# Managing underwater positions

In [21]:
import pandas as pd
import numpy as np
import logging
from backtesting import Strategy, Backtest


In [22]:
df = pd.read_csv('eth_hourly_data.csv', index_col=0, parse_dates=True)

In [23]:

df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,Quote volume,Trade count,Taker base volume,Taker quote volume,signal
Open time,,,,,,,,,,
2020-01-01 00:00:00+00:00,129.16,129.19,128.68,128.87,7769.17336,1.000930e+06,2504,4149.93345,5.346193e+05,2
2020-01-01 01:00:00+00:00,128.87,130.65,128.78,130.64,11344.65516,1.474278e+06,4885,5930.54276,7.704861e+05,2
2020-01-01 02:00:00+00:00,130.63,130.98,130.35,130.85,7603.35623,9.940256e+05,3046,3324.35218,4.346754e+05,1
2020-01-01 03:00:00+00:00,130.85,130.89,129.94,130.20,4968.55433,6.473610e+05,2818,1810.03564,2.358903e+05,1
2020-01-01 04:00:00+00:00,130.21,130.74,130.15,130.20,3397.90747,4.430067e+05,2264,1839.74371,2.398483e+05,2
...,...,...,...,...,...,...,...,...,...,...
2023-05-29 13:00:00+00:00,1898.20,1902.84,1895.80,1900.10,8232.89940,1.563808e+07,17187,3478.40000,6.607148e+06,2
2023-05-29 14:00:00+00:00,1900.09,1902.57,1885.00,1894.75,14439.17890,2.735379e+07,21559,6043.98480,1.144905e+07,1
2023-05-29 15:00:00+00:00,1894.74,1899.14,1886.31,1889.41,14199.72270,2.686451e+07,23161,7200.48690,1.362243e+07,1


## Generate a bunch of random entry signals

In [24]:
import numpy as np
df['signal'] = np.random.random(len(df))
df['signal'] = df['signal'].apply(lambda x: 1 if x<0.5 else 2) # 1 for buy, 2 for sell

In [25]:
df[df['signal']==1].count()+df[df['signal']==2].count()
#len(df)

Open                  29842
High                  29842
Low                   29842
Close                 29842
Volume                29842
Quote volume          29842
Trade count           29842
Taker base volume     29842
Taker quote volume    29842
signal                29842
dtype: int64

In [26]:
df.rename(columns={'Open time':'Datetime'}, inplace=True)

In [29]:

test_df = df # df.loc['2021'] if you want to test on a specific time period

def SIGNAL():
    return test_df.signal

# df.columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal']

class TradingDown(Strategy):
    # Set up some variables to be used by the strategy these can all be optimized
    initsize = .2 # Shown in decimal ie. 0.10 = 10% how much to buy on the first trade
    addsize = .1 # Shown in decimal ie. 0.10 = 10% how much to add to the position
    multiplier = .5 # For position sizing addsize times multiplier if the last trade was a loss
    max_number_of_trades = 400 # Maximum number of trades to have open at any one time

    take_profit = 0.01 # Shown in decimal ie. 0.10 = 10%
    position_loss_threshold = 0.02 # how much does the position have to be down before we add to it Shown in decimal ie. 0.10 = 10% 
    trade_loss_threshold = 0.04 # how much does the trade have to be down before we add to it Shown in decimal ie. 0.10 = 10%
    


    def position_sizer(self) -> float:
        '''
        This is the position sizing function. It will be called on every trade to determine how much to buy or sell
        Currently the logic is to buy XX% of the portfolio on the first trade, then add to the trade if the position is down
        if the last trade is down more than the trade_loss_threshold then add to the position by addsize*multiplier
        '''
        size = 0.01  # Initialize as just 1percent of the portfolio
        # If we have no position, then buy the initsize
        if self.position.size == 0:
            size = self.initsize
        # If we are not down on the position by more than the position_loss_threshold, then return a tiny amount
        # elif self.position.pl_pct > -self.position_loss_threshold:
        #     size = 0.01  # Return tiny amount just 1% of the portfolio
        # If we are down on the position by more than the position_loss_threshold, then add to the position
        elif (self.position.pl_pct < -self.position_loss_threshold and
            len(self.trades) < self.max_number_of_trades):
            if self.trades[-1].pl_pct < -self.trade_loss_threshold: # Check the last trade to see if we are up or down on it
                size = self.addsize*self.multiplier
                return size
            else: 
                # If you are down on the position but the last trade is profitable, then add to the position by a fraction of the addsize
                # Comment the first one out and replace it with the second one to be more conservative
                size = self.addsize * 0.5 # Add a reduced 50% of the addsize 
                # size=0.01

        return size


    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        # Create some variables to make the code easier to read and for logging
        current_cash = self._broker._cash
        current_margin_available = self._broker.margin_available
        current_position_value = self.position.size*self.data.Close[-1]
        logging.info(f"Current cash: {current_cash} Current margin available: {current_margin_available} \
                     Current position value: {current_position_value},  \
                     Current Position PnL: {self.position.pl_pct}")
        
        # Long signals
        # Opening first trade in a group
        if self.signal1==1 and len(self.trades)==0:   # if the signal is 2 and we have no trades open
            tp1 = self.data.Close[-1] + self.data.Close[-1]*self.take_profit # Set the take profit price
            self.buy(tp=tp1, size=self.position_sizer()) # add sl=sl1,
            # print(f'Just bought {self.orders[-1].size} total cost: {self.position_sizer()*self.data.Close[-1]}')

        # Adding to a group of trades if the position is down
        elif (
            # if we have a position
            self.position and 
            # if the number of trades is less than the max number of trades
            len(self.trades) < self.max_number_of_trades and 
            # if the entire position's profit/loss percentage is less than the loss threshold
            self.position.pl_pct < -self.position_loss_threshold and 
            # if the profit/loss percentage of the last trade is less than the loss threshold
            self.trades[-1].pl_pct < -self.trade_loss_threshold
            ): 
            
            # sl1 = self.data.Close[-1] - self.data.Close[-1]*self.stop_loss_take_profit
            tp1 = self.data.Close[-1] + self.data.Close[-1]*self.take_profit # Set the take profit price
            self.buy(size=self.position_sizer()) # add sl=sl1, tp=tp1
           
        # If the entire group of trades is profitable then close the entire position
        elif self.position.pl_pct > self.take_profit:
            self.position.close()
            logging.info(f'Closed position {self.position.size} with pnl of {self.position.pl} at bar {self.data.index[-1]}')
        

        # Short signals
        # elif self.signal1==2 and len(self.trades) < self.max_number_of_trades:         
        #     # sl1 = self.data.Close[-1] + self.data.Close[-1]*self.stop_loss_take_profit
        #     tp1 = self.data.Close[-1] - self.data.Close[-1]*self.stop_loss_take_profit
        #     self.sell(tp=tp1, size=self.mysize) # add sl=sl1,

In [31]:

bt = Backtest(test_df, 
              TradingDown,
              cash=100000, 
              trade_on_close=True, 
              commission=.00, 
              exclusive_orders=False, 
              margin=1, # Set this to 0.5 for 2x leverage, 0.25 for 4x leverage, 0.125 for 8x leverage, etc.
              )
stat = bt.run()

bt.plot()


/Users/ericervin/opt/anaconda3/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


TypeError: Index.get_loc() got an unexpected keyword argument 'method'

Now run it where resample is set to false

In [33]:
bt.plot(resample=False)

/Users/ericervin/opt/anaconda3/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ericervin/opt/anaconda3/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3788', ...)

Success, On my machine anyway.

# Now let's run the optimization
Note this isn't working on my machine. I believe I read somewhere in the issues that SKOPT hasn't been updated in a while and is causing a lot of the problems.

In [11]:
import numpy as np

stats_skopt, bt_heatmap, optimize_result = bt.optimize(
    initsize=[0.1, 0.2],
    addsize=[0.1, 0.3],
    take_profit= [0.01, 0.05],
    position_loss_threshold=[0.01, 0.05],
    trade_loss_threshold=[0.005, 0.05],
    max_number_of_trades=[5,50],
    multiplier=[1, 3],
    maximize='Equity Final [$]',
    # method='grid',
    method='skopt',
    max_tries=100,
    random_state=42,


)


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations